In [ ]:
!nvidia-smi

Sat Oct  8 19:32:43 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 432 kB 25.6 MB/s 
     |████████████████████████████████| 4.9 MB 58.2 MB/s 
     |████████████████████████████████| 115 kB 47.4 MB/s 
     |████████████████████████████████| 163 kB 51.2 MB/s 
     |████████████████████████████████| 212 kB 69.0 MB/s 
     |████████████████████████████████| 127 kB 74.0 MB/s 
     |████████████████████████████████| 6.6 MB 54.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
!cp drive/MyDrive/vtb_insights_data.json ./

In [ ]:
import json
import re
from glob import glob
from pathlib import Path

import datasets
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
import torch
import transformers
from datasets import load_dataset
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from transformers import (AutoTokenizer, AutoModelForSequenceClassification, 
                          TrainingArguments, Trainer)

In [ ]:
with open("vtb_insights_data.json") as f:
    data = json.load(f)

In [ ]:
df = pd.DataFrame(data, columns=["summary", "target"])
df = df[~df["summary"].isna()]

In [ ]:
df = df.drop_duplicates(subset=["summary"])
df["target"].value_counts()

0    947
1    116
Name: target, dtype: int64

In [ ]:
# df = df[df["summary"].str.len() >= 50]
# df["target"].value_counts()

In [ ]:
df.columns = ["text", "label"]

In [ ]:
df["text"] = df["text"].apply(lambda x: re.sub("<.*?>", "", x))

In [ ]:
# df = df[df["text"].str.len() >= 50]
# df["label"].value_counts()

0    12192
1      277
Name: label, dtype: int64

In [ ]:
df = df.drop_duplicates(subset=["text"])
df["label"].value_counts()

0    947
1    116
Name: label, dtype: int64

In [ ]:
train, test = train_test_split(df, test_size=0.2, random_state=42)
val, test = train_test_split(test, test_size=0.5, random_state=42)

In [ ]:
df_dataset = {
    "train": train,
    "val": val,
    "test": test,
}

In [ ]:
from datasets import Dataset

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        max_length=256,
        padding='max_length',
        truncation=True
    )

# model_ckpt = "bert-base-multilingual-uncased"
model_ckpt = "cointegrated/rubert-tiny2"
tokenizer = AutoTokenizer.from_pretrained(
    model_ckpt, problem_type="multi_label_classification")

hf_datasets = dict()
for key, value in df_dataset.items():
    dataset = Dataset.from_pandas(value, split="key")
    hf_datasets[key] = dataset.map(tokenize_function, batched=True)

Downloading:   0%|          | 0.00/401 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
num_labels = df_dataset["train"]["label"].unique().shape[0]
model = AutoModelForSequenceClassification.from_pretrained(
    model_ckpt,
    num_labels=num_labels,
    problem_type="single_label_classification").to('cuda')

Downloading:   0%|          | 0.00/715 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/118M [00:00<?, ?B/s]

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not 

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # print("pred", predictions)
    # print("label", labels)
    pred_labels = np.argmax(predictions, axis=1)
    f1_results = f1_score(labels,
                          pred_labels,
                          # (predictions > 0.5).astype(int),
                          average=None)
    f1_micro = f1_score(labels,
                        pred_labels,
                        average="micro")
    f1_macro = f1_score(labels,
                        pred_labels,
                        average="macro")
    output = {
        # 'accuracy_thresh': accuracy_thresh(predictions, labels),
        # 'f1': f1_results,
        "f1_micro": f1_micro,
        "f1_macro": f1_macro
        }
    # for k, v in label_dict.items():
    #     output[f"f1_{k}"] = f1_results[v]
    return output

In [ ]:
from transformers import EarlyStoppingCallback
batch_size = 256

args = TrainingArguments(
    output_dir="multiclass_model",
    save_strategy = "epoch",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=100,
    weight_decay=0.01,
    save_total_limit=5,
   load_best_model_at_end=True
)


trainer = Trainer(
    model,
    args,
    train_dataset=hf_datasets["train"],
    eval_dataset=hf_datasets["val"],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(4)],
)

trainer.evaluate()
trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, __index_level_0__. If text, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 106
  Batch size = 256


The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, __index_level_0__. If text, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 850
  Num Epochs = 100
  Instantaneous batch size per device = 256
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 1
  Total optimization steps = 400


Epoch,Training Loss,Validation Loss,F1 Micro,F1 Macro
1,No log,0.453391,0.877358,0.467337
2,No log,0.424598,0.877358,0.467337
3,No log,0.401845,0.877358,0.467337
4,No log,0.385323,0.877358,0.467337
5,No log,0.374903,0.877358,0.467337
6,No log,0.369084,0.877358,0.467337
7,No log,0.365445,0.877358,0.467337
8,No log,0.362705,0.877358,0.467337
9,No log,0.359834,0.877358,0.467337
10,No log,0.356308,0.877358,0.467337


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, __index_level_0__. If text, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 106
  Batch size = 256
Saving model checkpoint to multiclass_model/checkpoint-4
Configuration saved in multiclass_model/checkpoint-4/config.json
Model weights saved in multiclass_model/checkpoint-4/pytorch_model.bin
tokenizer config file saved in multiclass_model/checkpoint-4/tokenizer_config.json
Special tokens file saved in multiclass_model/checkpoint-4/special_tokens_map.json
Deleting older checkpoint [multiclass_model/checkpoint-24] due to args.save_total_limit
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, __index_level_0__. If text, __inde

KeyboardInterrupt: ignored

In [ ]:
trainer.save_model("vtb_insight_model")

Saving model checkpoint to vtb_insight_model
Configuration saved in vtb_insight_model/config.json
Model weights saved in vtb_insight_model/pytorch_model.bin
tokenizer config file saved in vtb_insight_model/tokenizer_config.json
Special tokens file saved in vtb_insight_model/special_tokens_map.json


In [ ]:
!zip vtb_insight_model.zip vtb_insight_model/*

  adding: vtb_insight_model/config.json (deflated 49%)
  adding: vtb_insight_model/pytorch_model.bin (deflated 8%)
  adding: vtb_insight_model/special_tokens_map.json (deflated 42%)
  adding: vtb_insight_model/tokenizer_config.json (deflated 43%)
  adding: vtb_insight_model/tokenizer.json (deflated 73%)
  adding: vtb_insight_model/training_args.bin (deflated 48%)
  adding: vtb_insight_model/vocab.txt (deflated 64%)


In [ ]:
cp vtb_insight_model.zip drive/MyDrive/

In [ ]:
ls -lah drive/MyDrive/vtb*

-rw------- 1 root root 104M Oct  8 09:54 drive/MyDrive/vtb_model2.zip
-rw------- 1 root root 104M Oct  8 09:19 drive/MyDrive/vtb_model.zip
-rw------- 1 root root 5.4M Oct  7 22:07 drive/MyDrive/vtb_news_hack.zip


In [ ]:
from transformers import pipeline

In [ ]:
!cp drive/MyDrive/vtb_model2.zip ./

In [ ]:
!unzip vtb_model2.zip

Archive:  vtb_model.zip
  inflating: vtb_model/config.json   
  inflating: vtb_model/pytorch_model.bin  
  inflating: vtb_model/special_tokens_map.json  
  inflating: vtb_model/tokenizer_config.json  
  inflating: vtb_model/tokenizer.json  
  inflating: vtb_model/training_args.bin  
  inflating: vtb_model/vocab.txt     


In [ ]:
pipe = pipeline("text-classification", "vtb_insight_model", device=0)

loading configuration file vtb_insight_model/config.json
Model config BertConfig {
  "_name_or_path": "vtb_insight_model",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "emb_size": 312,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 312,
  "initializer_range": 0.02,
  "intermediate_size": 600,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 2048,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 3,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.22.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 83828
}

loading configuration file vtb_insight_model/config.json
Model config BertConfig {
  "_name_or_path": "vtb_insight_model",
  "architectures": [
    "BertForSequenceClassific

In [ ]:
pipe("Украина")

[{'label': 'LABEL_0', 'score': 0.985740602016449}]

In [ ]:
pipe("бухгалтер")

[{'label': 'LABEL_0', 'score': 0.9819790124893188}]

In [ ]:
tokenizer_kwargs = {'padding':True,'truncation':True,'max_length':512}

for i, text in enumerate(df[df["label"] == 0]["text"].values):
    label = pipe(text, **tokenizer_kwargs)[0]["label"]
    if label == "LABEL_1":
        print(text)
        print("-" * 80)

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:1046: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


Легендарный инвестор Рэй Далио покинул пост главы собственной инвесткомпании Bridgewater Associates, основанной им 47 лет назад и специализирующейся на управлении хедж-фондами.
--------------------------------------------------------------------------------
В каждом зале будут собраны профессионалы соответствующей области, среди которых вице-президент Ассоциации банков России Алексей Войлуков, директор по стратегии ИК «Финам» Ярослав Кабаков, главный инвестиционный директор Huawei в России Григорий Лещенко, руководитель Kaspersky Product Studio Николай Майданчик, председатель совета по финансово-промышленной и инвестиционной политике, член правления Торгово-промышленной палаты РФ Владимир Гамза.В завершение вечера состоится церемония награждения лауреатов премии Investment Leaders 2022.
--------------------------------------------------------------------------------
Согласно заявлению, это первый ЦФА в стране на на диверсифицированный портфель активов, который содержит готовую инвестид

In [ ]:
tokenizer_kwargs = {'padding':True,'truncation':True,'max_length':512}

for i, text in enumerate(df[df["label"] == 0]["text"].values):
    label = pipe(text, **tokenizer_kwargs)[0]["label"]
    if label == "LABEL_1":
        print(i, text)

106 Легендарный инвестор Рэй Далио покинул пост главы собственной инвесткомпании Bridgewater Associates, основанной им 47 лет назад и специализирующейся на управлении хедж-фондами.
506 В каждом зале будут собраны профессионалы соответствующей области, среди которых вице-президент Ассоциации банков России Алексей Войлуков, директор по стратегии ИК «Финам» Ярослав Кабаков, главный инвестиционный директор Huawei в России Григорий Лещенко, руководитель Kaspersky Product Studio Николай Майданчик, председатель совета по финансово-промышленной и инвестиционной политике, член правления Торгово-промышленной палаты РФ Владимир Гамза.В завершение вечера состоится церемония награждения лауреатов премии Investment Leaders 2022.
544 Согласно заявлению, это первый ЦФА в стране на на диверсифицированный портфель активов, который содержит готовую инвестидею.


In [ ]:
preds = []

for text in df_dataset["val"]["text"].values:
    preds.append(pipe(text, **tokenizer_kwargs))

In [ ]:
preds[0]

In [ ]:
processed_preds = [int(p[0]["label"][6:]) for p in preds]
processed_proba = [p[0]["score"] for p in preds]

In [ ]:
def check_metrics(  true_y,
                    pred_y,
                    pred_y_proba,
                    threshold):
        assert len(true_y) == len(pred_y)

        bad_answers = []
        accuracy = []

        for i in range(len(true_y)):

            if pred_y[i] != true_y[i] and pred_y_proba[i] > threshold:
                # F sure(FP)
                bad_answers.append(1)

            if pred_y[i] == true_y[i] and pred_y_proba[i] > threshold:
                accuracy.append(1)

        return (np.around([sum(accuracy) / len(true_y) * 100], 5)[0],
                np.around(sum(bad_answers) / len(true_y) * 100, 5)
                )

In [ ]:
for threshold in (0.4, 0.5, 0.6, 0.7,
                  0.8, 0.85, 0.86, 0.87, 0.88, 0.89,
                  0.9, 0.91, 0.92, 0.95, 0.98):
    metrics = check_metrics(df_dataset["val"]["labels"].values,
                            processed_preds,
                            processed_proba,
                            threshold)
    print(threshold, metrics)
    